## Run twins test (use another shell terminal)
Run twins test and output log to a temp file by this
`cargo xtest -p consensus twins_proposer_test -- --nocapture 2> /tmp/libra.log`

In [ ]:
import re
import json
from collections import Counter
from collections import defaultdict

#twin_pattern = re.compile(r'TwinId\s*{\s*id\s*:\s*(\S*)\s*,\s*author\s*:\s*(\S*)\s*}')
qc_pattern = re.compile(r'([0-9]+)-node-twins.*({"quorum_cert":.*})')
#commit_pattern = re.compile(r'([0-9]+)-node-twins.*({"commit_qc_proposed_id":.*})')

## parse the forensic log of twins test

In [ ]:
libra_twins_forensic_log = '/tmp/libra.log'

twin_nodes = {}
qcs = defaultdict(dict) # block hash -> qc
commit_qcs = defaultdict(dict) # block id -> grandparent id (the commit block)

# with open(libra_twins_forensic_log) as fin:
#     for line in fin:
#         m = twin_pattern.search(line)
#         if m is not None:
#             twin_nodes[m.group(1)]=m.group(2)
#             qcs[m.group(1)]={}
#             commit_qcs[m.group(1)]={}
with open(libra_twins_forensic_log) as fin:
    for line in fin:
        m = qc_pattern.search(line)
        if m is not None:
            d = json.loads(m.group(2))
            r = d["quorum_cert"]["vote_data"]["proposed"]["round"] # round/view
            h = d["quorum_cert"]["vote_data"]["proposed"]["id"] # block hash/ proposed id
#             p = d["quorum_cert"]["vote_data"]["parent"]["id"] # parent hash
            grand_h = d["quorum_cert"]["signed_ledger_info"]["V0"]["ledger_info"]["commit_info"]["id"] # grandparent hash/ commit id
            qcs[m.group(1)][h]=d["quorum_cert"]
#         m = commit_pattern.search(line)
#         if m is not None:
#             d = json.loads(m.group(2))
#             commit_qcs[m.group(1)][d["commit_qc_proposed_id"]]=d["commit_qc_commit_id"]


## output all qcs
(for now we can check if there is disagreement by hand...)

In [ ]:
for i in range(6):
    a=str(i)
    for _qc in qcs[a].values():
        r = _qc["vote_data"]["proposed"]["round"]
        h = _qc["vote_data"]["proposed"]["id"]
        print('node {} qc for {} at round {}'.format(i,h[:6],r))

In [ ]:
def hotstuff_forensic_within_view(qc_1, qc_2):
    epoch_1 = qc_1["vote_data"]["proposed"]["epoch"]
    epoch_2 = qc_2["vote_data"]["proposed"]["epoch"]
    round_1 = qc_1["vote_data"]["proposed"]["round"]
    round_2 = qc_2["vote_data"]["proposed"]["round"]
    id_1 = qc_1["signed_ledger_info"]["V0"]["ledger_info"]["commit_info"]["id"]
    id_2 = qc_2["signed_ledger_info"]["V0"]["ledger_info"]["commit_info"]["id"]
    assert epoch_1 == epoch_2
    assert round_1 == round_2
    assert id_1 != id_2
    # omit the signature checking
    signers_1 = qc_1["signed_ledger_info"]["V0"]["signatures"]
    signers_2 = qc_2["signed_ledger_info"]["V0"]["signatures"]
    signers_1 = set(signers_1.keys())
    signers_2 = set(signers_2.keys())
    return signers_1.intersection(signers_2)

In [ ]:
def hotstuff_forensic_across_views(qc_1, qc_2, pre_qc2_qcs):
    epoch_1 = qc_1["vote_data"]["proposed"]["epoch"]
    epoch_2 = qc_2["vote_data"]["proposed"]["epoch"]
    round_1 = qc_1["vote_data"]["proposed"]["round"]
    round_2 = qc_2["vote_data"]["proposed"]["round"]
    id_1 = qc_1["signed_ledger_info"]["V0"]["ledger_info"]["commit_info"]["id"]
    id_2 = qc_2["signed_ledger_info"]["V0"]["ledger_info"]["commit_info"]["id"]
    assert epoch_1 == epoch_2
    assert round_1 < round_2
    # omit checking pre_qc2_qcs is a valid chain of qc ending with qc_2
    # omit checking qc_1 and qc_2 are indeed in different branch
    # omit the signature checking
    signers_1 = qc_1["signed_ledger_info"]["V0"]["signatures"]
    round2qc = {}
    for _qc in pre_qc2_qcs:
        round2qc[_qc["vote_data"]["proposed"]["round"]] = _qc
    for i in range(round_1, round_2+1):
        if i in round2qc:
            _qc = round2qc[i]
            _round = _qc["vote_data"]["parent"]["round"]
            _id = _qc["vote_data"]["parent"]["id"]
            assert _round < round_1 or _round == round_1 and _id != id_1, "the first higher qc should violate safety voting rule"
            signers_2 = _qc["signed_ledger_info"]["V0"]["signatures"]
            signers_1 = set(signers_1.keys())
            signers_2 = set(signers_2.keys())
            return signers_1.intersection(signers_2)
    assert false, "should not reach here"

we can extract the qc by hand, and use them to function `hotstuff_forensic_within_view` or `hotstuff_forensic_across_views`

In [ ]:
qc_1 = None
qc_2 = None
for _qc in qcs["2"].values():
    if _qc["signed_ledger_info"]["V0"]["ledger_info"]["commit_info"]["round"]==1:
        qc_1=_qc
for _qc in qcs["3"].values():
    if _qc["signed_ledger_info"]["V0"]["ledger_info"]["commit_info"]["round"]==1:
        qc_2=_qc
print(hotstuff_forensic_within_view(qc_1,qc_2))

In [ ]:
qc_1 = None
qc_2 = None
for _qc in qcs["2"].values():
    if _qc["signed_ledger_info"]["V0"]["ledger_info"]["commit_info"]["round"]==1:
        qc_1=_qc
for _qc in qcs["5"].values():
    if _qc["signed_ledger_info"]["V0"]["ledger_info"]["commit_info"]["round"]==2:
        qc_2=_qc
print(hotstuff_forensic_across_views(qc_1,qc_2,[qc_2]))

In [ ]:
# print("list of accounts: (account who appears more than once is set to be corrupted by twins)\t", Counter(twin_nodes.values()))

# # the two commit qcs from node 2 and 3
# node2_commit = list(commit_qcs["2"].keys())[0]
# node3_commit = list(commit_qcs["3"].keys())[0]
# forensic_report = hotstuff_forensic_within_view(qcs["2"][node2_commit], qcs["3"][node3_commit])
# print("forensic report: culprits are", forensic_report)